In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/list-of-bad-words/full-list-of-bad-words_text-file_2022_05_05.txt
/kaggle/input/google-profanity/googleprofanity(1000).txt
/kaggle/input/profanities-in-english-collection/profanity_en.csv
/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip


In [2]:
!pip install transformers torch sklearn


  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://g

In [75]:
df=pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip')
df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0


In [4]:
# df['__label__1']=df.loc[:,'toxic':'identity_hate'].sum(axis=1).gt(0).astype(int)
# df['__label__1']

In [5]:
# df['__label__2']=(df.loc[:,'toxic':'identity_hate'].sum(axis=1)==0).astype(int)
# df['__label__2']

In [76]:
df['label']=df.loc[:,'toxic':'identity_hate'].sum(axis=1).gt(0).astype(int)
df['label']


0         0
1         0
2         0
3         0
4         0
         ..
159566    0
159567    0
159568    0
159569    0
159570    0
Name: label, Length: 159571, dtype: int64

In [77]:
df=df.drop(['toxic', 'severe_toxic','obscene','threat', 'insult', 'identity_hate'], axis=1)

In [78]:
add_data= pd.read_csv('/kaggle/input/google-profanity/googleprofanity(1000).txt', sep='\t',header=None)
add_data.columns = ['comment_text']
add_data['label']=1
add_data

,comment_text,label
0,2 girls 1 cup,1
1,2g1c,1
2,4r5e,1
3,5h1t,1
4,5hit,1
...,...,...
953,yaoi,1
954,yellow showers,1
955,yiffy,1
956,zoophilia,1


In [79]:
df=pd.concat([df,add_data],axis=0)
df

,id,comment_text,label
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0
...,...,...,...
953,NaN,yaoi,1
954,NaN,yellow showers,1
955,NaN,yiffy,1
956,NaN,zoophilia,1


In [80]:
import re
# Compile regex patterns
pattern_punctuation = re.compile(r'[^\w\s]')
pattern_whitespace = re.compile(r'\s+')


In [82]:
# Vectorized operations
df['comment_text'] = df['comment_text'].str.replace(pattern_punctuation, '', regex=True)  # Remove punctuation
df['comment_text'] = df['comment_text'].str.replace(pattern_whitespace, ' ', regex=True)  # Replace multiple whitespace with single space
df['comment_text'] = df['comment_text'].str.strip()  # Remove leading and trailing whitespace
df['comment_text']=df['comment_text'].str.lower()

In [83]:
df

,id,comment_text,label
0,0000997932d777bf,explanation why the edits made under my userna...,0
1,000103f0d9cfb60f,daww he matches this background colour im seem...,0
2,000113f07ec002fd,hey man im really not trying to edit war its j...,0
3,0001b41b1c6bb37e,more i cant make any real suggestions on impro...,0
4,0001d958c54c6e35,you sir are my hero any chance you remember wh...,0
...,...,...,...
953,NaN,yaoi,1
954,NaN,yellow showers,1
955,NaN,yiffy,1
956,NaN,zoophilia,1


import spacy
# Load spaCy English language model
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Define preprocessing function using spaCy
def preprocess(text):
    doc = nlp(text)
    # Lemmatize and remove stop words
    lemmatized_text = ' '.join([token.lemma_ for token in doc if not token.is_stop])
    return lemmatized_text

# Apply the preprocessing function
df['comment_text'] = df['comment_text'].apply(preprocess)


In [14]:

#import spacy
# Load spaCy English language model
#nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Define preprocessing function using spaCy
#def preprocess(text):
#    doc = nlp(text)
    # Lemmatize and remove stop words
#    lemmatized_text = ' '.join([token.lemma_ for token in doc if not token.is_stop])
#    return lemmatized_text

# Apply the preprocessing function
#df['processed_text'] = df['comment_text'].apply(preprocess)


In [84]:
df

,id,comment_text,label
0,0000997932d777bf,explanation why the edits made under my userna...,0
1,000103f0d9cfb60f,daww he matches this background colour im seem...,0
2,000113f07ec002fd,hey man im really not trying to edit war its j...,0
3,0001b41b1c6bb37e,more i cant make any real suggestions on impro...,0
4,0001d958c54c6e35,you sir are my hero any chance you remember wh...,0
...,...,...,...
953,NaN,yaoi,1
954,NaN,yellow showers,1
955,NaN,yiffy,1
956,NaN,zoophilia,1


# XLNet

In [17]:
# import pandas as pd
# import torch
# from transformers import XLNetTokenizer, XLNetForSequenceClassification, Trainer, TrainingArguments
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score, precision_recall_fscore_support


2024-07-11 08:01:38.825044: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-11 08:01:38.825166: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-11 08:01:38.930559: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [19]:

# from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
# from nltk.stem import WordNetLemmatizer
# from nltk.corpus import wordnet
# from nltk import pos_tag
# import nltk

# # Download necessary NLTK data
# nltk.download('wordnet')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('omw-1.4')



# # Function to get the part of speech tag for lemmatization
# def get_wordnet_pos(word):
#     """Map POS tag to first character lemmatize() accepts"""
#     tag = pos_tag([word])[0][1][0].upper()
#     tag_dict = {"J": wordnet.ADJ,
#                 "N": wordnet.NOUN,
#                 "V": wordnet.VERB,
#                 "R": wordnet.ADV}
#     return tag_dict.get(tag, wordnet.NOUN)

# # Initialize lemmatizer
# lemmatizer = WordNetLemmatizer()

# # Function to preprocess text
# def preprocess(text):
#     # Remove punctuation
#     text = re.sub(r'[^\w\s]', '', text)
#     # Remove multiple whitespace
#     text = re.sub(r'\s+', ' ', text).strip()
#     # Tokenize and remove stopwords
#     tokens = [word for word in text.split() if word.lower() not in ENGLISH_STOP_WORDS]
#      #Lemmatize tokens
#     lemmatized_tokens = [lemmatizer.lemmatize(token, get_wordnet_pos(token)) for token in tokens]
#     return ' '.join(lemmatized_tokens)

# # Apply preprocessing function
# df['clean_text'] = df['comment_text'].apply(preprocess)

# print(df)


In [20]:
!pip install nltk

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


# Is Actually Helpful

In [85]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

import re

nltk.data.path.append("/kaggle/working/")
# Ensure necessary NLTK data is downloaded
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('corpora')
import nltk

nltk.download('wordnet')


# Now you can use WordNet functionalities
from nltk.corpus import wordnet
import nltk
nltk.download('wordnet')

from nltk.corpus import wordnet

# Load data
data = df.copy()

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()
# Tokenize text
data['tokens'] = data['comment_text'].apply(word_tokenize)

# Remove stop words
stop_words = set(stopwords.words('english'))
data['tokens'] = data['tokens'].apply(lambda x: [word for word in x if word not in stop_words])

# Lemmatize tokens
data['tokens'] = data['tokens'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

# Join tokens back to a single string
data['processed_text'] = data['tokens'].apply(lambda x: ' '.join(x))

# TF-IDF features
tfidf = TfidfVectorizer(max_features=5000)
tfidf_features = tfidf.fit_transform(data['processed_text']).toarray()
tfidf_df = pd.DataFrame(tfidf_features, columns=tfidf.get_feature_names_out())
data = pd.concat([data, tfidf_df], axis=1)

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Error loading corpora: Package 'corpora' not found in
[nltk_data]     index
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


LookupError: 
**********************************************************************
  Resource 'corpora/wordnet' not found.  Please use the NLTK
  Downloader to obtain the resource:  >>> nltk.download()
  Searched in:
    - '/root/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - '/kaggle/working/'
    - '/kaggle/working/'
**********************************************************************

In [86]:
data

,id,comment_text,label,tokens
0,0000997932d777bf,explanation why the edits made under my userna...,0,"[explanation, edits, made, username, hardcore,..."
1,000103f0d9cfb60f,daww he matches this background colour im seem...,0,"[daww, matches, background, colour, im, seemin..."
2,000113f07ec002fd,hey man im really not trying to edit war its j...,0,"[hey, man, im, really, trying, edit, war, guy,..."
3,0001b41b1c6bb37e,more i cant make any real suggestions on impro...,0,"[cant, make, real, suggestions, improvement, w..."
4,0001d958c54c6e35,you sir are my hero any chance you remember wh...,0,"[sir, hero, chance, remember, page, thats]"
...,...,...,...,...
953,NaN,yaoi,1,[yaoi]
954,NaN,yellow showers,1,"[yellow, showers]"
955,NaN,yiffy,1,[yiffy]
956,NaN,zoophilia,1,[zoophilia]


In [87]:
data[['comment_text','label','tokens']].to_csv("cleaned_dataset", index=False)

# Feature Extraction

In [88]:
data.label

0      0
1      0
2      0
3      0
4      0
      ..
953    1
954    1
955    1
956    1
957    1
Name: label, Length: 160529, dtype: int64

In [89]:
 texts = data['comment_text'].tolist()  
 labels = df['label'].tolist()

In [90]:

train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2)


In [27]:
# from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
# import torch
# # Load tokenizer and model
# tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
# model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(set(labels)))

# # Tokenize data
# train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
# val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

# # Convert to PyTorch datasets
# class Dataset(torch.utils.data.Dataset):
#     def __init__(self, encodings, labels):
#         self.encodings = encodings
#         self.labels = labels

#     def __getitem__(self, idx):
#         item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
#         item['labels'] = torch.tensor(self.labels[idx])
#         return item

#     def __len__(self):
#         return len(self.labels)

# train_dataset = Dataset(train_encodings, train_labels)
# val_dataset = Dataset(val_encodings, val_labels)

# # # Define training arguments
# # training_args = TrainingArguments(
# #     output_dir='./results',
# #     num_train_epochs=3,
# #     per_device_train_batch_size=8,
# #     per_device_eval_batch_size=8,
# #     warmup_steps=500,
# #     weight_decay=0.01,
# #     logging_dir='./logs',
# #     logging_steps=10,
# #     evaluation_strategy="epoch"
# # )

# training_args = TrainingArguments(
#     output_dir='/kaggle/working/',
#     num_train_epochs=3,
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     warmup_steps=500,
#     weight_decay=0.01,
#     logging_dir='/kaggle/working/',
#     logging_steps=10,
#     evaluation_strategy="epoch",
#     save_strategy="epoch",  # Save checkpoints only at the end of each epoch
#     save_total_limit=1,      # Keep only the last checkpoint
#     report_to="none"         # Disable wandb
# )

# # Define metrics function
# def compute_metrics(pred):   
#     labels = pred.label_ids
#     preds = pred.predictions.argmax(-1)
#     precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
#     acc = accuracy_score(labels, preds)
#     return {
#         'accuracy': acc,
#         'f1': f1,
#         'precision': precision,
#         'recall': recall
#     }

# # Create Trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#     compute_metrics=compute_metrics
# )

# # # Train the model
# # trainer.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# training_args = TrainingArguments(
#     output_dir='/kaggle/working/',
#     num_train_epochs=3,
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     warmup_steps=500,
#     weight_decay=0.01,
#     logging_dir='/kaggle/working/',
#     logging_steps=10,
#     evaluation_strategy="epoch",
#     save_strategy="epoch",  # Save checkpoints only at the end of each epoch
#     save_total_limit=1,      # Keep only the last checkpoint
#     report_to="none"         # Disable wandb
# )


# (Training on GPU)

In [ ]:
import os
os.environ['WANDB_DISABLED'] = 'true'

from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# Load tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(set(labels)))

# Tokenize data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

# Convert to PyTorch datasets
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = Dataset(train_encodings, train_labels)
val_dataset = Dataset(val_encodings, val_labels)

# Define training arguments
training_args = TrainingArguments(
    output_dir='/kaggle/working/',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='/kaggle/working/',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",  # Save checkpoints only at the end of each epoch
    save_total_limit=1,     # Keep only the last checkpoint
    report_to="none"        # Disable wandb
)

# Define metrics function
def compute_metrics(pred):   
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# # Train the model
# trainer.train()
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()



Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.017900,0.133622,0.964562,0.963996,0.963656,0.964562


In [91]:
import os
os.environ['WANDB_DISABLED'] = 'true'

from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# Load tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(set(labels)))

# Tokenize data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

# Convert to PyTorch datasets
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = Dataset(train_encodings, train_labels)
val_dataset = Dataset(val_encodings, val_labels)

# Define training arguments
training_args = TrainingArguments(
    output_dir='/kaggle/working/',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='/kaggle/working/',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",  # Save checkpoints only at the end of each epoch
    save_total_limit=1,     # Keep only the last checkpoint
    report_to="none"        # Disable wandb
)

# Define metrics function
def compute_metrics(pred):   
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# # Train the model
# trainer.train()
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()



Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.136400,0.135708,0.961565,0.961272,0.961041,0.961565
2,0.007100,0.162036,0.964212,0.962590,0.963077,0.964212
3,0.033100,0.151891,0.964991,0.964370,0.964085,0.964991


TrainOutput(global_step=48159, training_loss=0.11059915130697588, metrics={'train_runtime': 12046.9328, 'train_samples_per_second': 31.981, 'train_steps_per_second': 3.998, 'total_flos': 5.103558221279846e+16, 'train_loss': 0.11059915130697588, 'epoch': 3.0})

In [93]:
# Save the model
model_path = "/kaggle/working/"
trainer.save_model(model_path)


In [95]:
import shutil

# Create a zip file of the model directory
shutil.make_archive('/kaggle/working/model_checkpoint_d', 'zip', '/kaggle/working/checkpoint-48159')


'/kaggle/working/model_checkpoint_d.zip'

In [97]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import torch

# Load tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Load model using config.json and model.safetensors
model_path = "/kaggle/working/checkpoint-48159"  # Directory containing config.json and model.safetensors
model = DistilBertForSequenceClassification.from_pretrained(model_path)

# Function to predict the class of a given text
def predict(text):
    # Tokenize input text
    inputs = tokenizer(text, truncation=True, padding=True, max_length=512, return_tensors="pt")

    # Forward pass through the model
    with torch.no_grad():
        outputs = model(**inputs)

    # Get predicted class
    logits = outputs.logits
    predicted_class = logits.argmax().item()

    return predicted_class

# Example text
text = "dick"

# Make prediction
predicted_class = predict(text)
print(f"Predicted class: {predicted_class}")


Predicted class: 1


In [98]:
text = "dumb"

# Make prediction
predicted_class = predict(text)
print(f"Predicted class: {predicted_class}")

Predicted class: 1


In [99]:
test=['ass', 'fucker', 'dum', 'butt', 'sick', 'love', 'cum', 'creampie']
for text in test:
    predicted_class = predict(text)
    print(f"Predicted class: {predicted_class}")

Predicted class: 1
Predicted class: 1
Predicted class: 1
Predicted class: 1
Predicted class: 1
Predicted class: 0
Predicted class: 1
Predicted class: 1


In [100]:
test=['sucker', 'dirt', 'pus', 'penis', 'nude']
for text in test:
    predicted_class = predict(text)
    print(f"Predicted class: {predicted_class}")

Predicted class: 1
Predicted class: 1
Predicted class: 1
Predicted class: 1
Predicted class: 1


In [101]:
test=['orgasm', 'cunt', 'cameltoe', 'lesbian', 'gay', 'virgin', 'pornstar','slut']
for text in test:
    predicted_class = predict(text)
    print(f"Predicted class: {predicted_class}")

Predicted class: 1
Predicted class: 1
Predicted class: 1
Predicted class: 1
Predicted class: 1
Predicted class: 1
Predicted class: 1
Predicted class: 1


In [102]:
test=['this is damn hot girl', 'i like the way u kiss me', 'Your skin is soft as silk', 'Your shirt is hot as fuck', 'motherfucker get away from my way', 'motherfvcker', 'i dont give a shit to pornhub','You are so cool']
for text in test:
    predicted_class = predict(text)
    print(f" {text} is {predicted_class}")

 this is damn hot girl is 1
 i like the way u kiss me is 0
 Your skin is soft as silk is 0
 Your shirt is hot as fuck is 1
 motherfucker get away from my way is 1
 motherfvcker is 1
 i dont give a shit to pornhub is 1
 You are so cool is 0


In [103]:
test=['gonna ride on this bitch', 'crazy people', 'you nigga', 'dicklicker', 'your puss', 'motherfvcker', 'you  bustard ugly fatso','boob']
for text in test:
    predicted_class = predict(text)
    print(f" {text} is {predicted_class}")

 gonna ride on this bitch is 1
 crazy people is 1
 you nigga is 1
 dicklicker is 1
 your puss is 1
 motherfvcker is 1
 you  bustard ugly fatso is 1
 boob is 1


In [104]:
test=['anal', 'gangbang', 'balllicker', 'bangbros', 'barenaked', 'bigtits', 'bi-sexual','cock', 'brestjob', 'bullshit', 'clit','clitorius','dildo','doggie-style','freak','jizz','nigro','piss','seduce', 'semen','masterbate','murder','panties','pee','undress', 'upskirt']
for text in test:
    predicted_class = predict(text)
    print(f" {text} is {predicted_class}")

 anal is 1
 gangbang is 1
 balllicker is 1
 bangbros is 1
 barenaked is 1
 bigtits is 1
 bi-sexual is 1
 cock is 1
 brestjob is 1
 bullshit is 1
 clit is 1
 clitorius is 1
 dildo is 1
 doggie-style is 1
 freak is 1
 jizz is 1
 nigro is 1
 piss is 1
 seduce is 1
 semen is 1
 masterbate is 1
 murder is 1
 panties is 1
 pee is 1
 undress is 1
 upskirt is 1


In [105]:
test=['gonna ride on this b!tch', 'you bastard', 'you ugly little fatso ', 'murder', 'gonna have fun with hot babes', 'f4ck', 'seduce','Homo Sapiens']
for text in test:
    predicted_class = predict(text)
    print(f" {text} is {predicted_class}")

 gonna ride on this b!tch is 0
 you bastard is 1
 you ugly little fatso  is 1
 murder is 1
 gonna have fun with hot babes is 0
 f4ck is 1
 seduce is 1
 Homo Sapiens is 1


In [106]:
test=['drug', 'virgin', 'feeling lusty', 'My libido is very high', 'butthole', 'shemale', 'i ride a bicycle','married couple having fun']
for text in test:
    predicted_class = predict(text)
    print(f" {text} is {predicted_class}")

 drug is 0
 virgin is 1
 feeling lusty is 0
 My libido is very high is 0
 butthole is 1
 shemale is 1
 i ride a bicycle is 0
 married couple having fun is 0


# more testing 

In [67]:
import pandas as pd

# Replace 'path_to_testdata.txt' with the actual path to your test data file
testdata_df = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip')

testdata_df


,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.
...,...,...
153159,fffcd0960ee309b5,". \n i totally agree, this stuff is nothing bu..."
153160,fffd7a9a6eb32c16,== Throw from out field to home plate. == \n\n...
153161,fffda9e8d6fafa9e,""" \n\n == Okinotorishima categories == \n\n I ..."
153162,fffe8f1340a79fc2,""" \n\n == """"One of the founding nations of the..."


In [68]:
# Vectorized operations
testdata_df['comment_text'] = testdata_df['comment_text'].str.replace(pattern_punctuation, '', regex=True)  # Remove punctuation
testdata_df['comment_text'] = testdata_df['comment_text'].str.replace(pattern_whitespace, ' ', regex=True)  # Replace multiple whitespace with single space
testdata_df['comment_text'] = testdata_df['comment_text'].str.strip()  # Remove leading and trailing whitespace


In [69]:
test_labels = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip')
test_labels

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...
153159,fffcd0960ee309b5,-1,-1,-1,-1,-1,-1
153160,fffd7a9a6eb32c16,-1,-1,-1,-1,-1,-1
153161,fffda9e8d6fafa9e,-1,-1,-1,-1,-1,-1
153162,fffe8f1340a79fc2,-1,-1,-1,-1,-1,-1


> **test_labels has some vulnerability**

# Train on TPU

In [ ]:
# import os
# import tensorflow as tf
# from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification, create_optimizer
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import precision_recall_fscore_support, accuracy_score
# import pandas as pd

# # Disable W&B
# os.environ['WANDB_DISABLED'] = 'true'

# # Load data

# texts = data['comment_text'].tolist()
# labels = df['label'].tolist()

# # Split data
# train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2)

# # Load tokenizer and model
# tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
# train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
# val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

# # Convert to TensorFlow datasets
# def create_tf_dataset(encodings, labels):
#     def gen():
#         for i in range(len(labels)):
#             yield {key: tf.convert_to_tensor(val[i], dtype=tf.int32) for key, val in encodings.items()}, tf.convert_to_tensor(labels[i], dtype=tf.int32)
#     return tf.data.Dataset.from_generator(gen, ({key: tf.int32 for key in encodings.keys()}, tf.int32))

# train_dataset = create_tf_dataset(train_encodings, train_labels)
# val_dataset = create_tf_dataset(val_encodings, val_labels)

# # Detect TPU
# try:
#     tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
#     print('Running on TPU ', tpu.master())
# except ValueError:
#     raise BaseException('ERROR: Not connected to a TPU runtime; please check your settings.')

# # Connect to the TPU cluster
# tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)

# # Create TPU strategy
# strategy = tf.distribute.TPUStrategy(tpu)

# # Define metrics function
# def compute_metrics(pred):   
#     labels = pred.label_ids
#     preds = tf.argmax(pred.predictions, axis=-1)
#     precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
#     acc = accuracy_score(labels, preds)
#     return {
#         'accuracy': acc,
#         'f1': f1,
#         'precision': precision,
#         'recall': recall
#     }

# # Train the model within TPU strategy scope
# with strategy.scope():
#     model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(set(labels)))
    
#     # Define optimizer
#     steps_per_epoch = len(train_dataset) // 8
#     num_train_steps = steps_per_epoch * 3
#     optimizer, _ = create_optimizer(init_lr=3e-5, num_train_steps=num_train_steps, num_warmup_steps=500)

#     model.compile(optimizer=optimizer, 
#                   loss=model.compute_loss, 
#                   metrics=['accuracy'])
    
#     model.fit(train_dataset.batch(8), epochs=3, validation_data=val_dataset.batch(8))

# # Evaluate the model
# results = model.evaluate(val_dataset.batch(8))
# print("Validation loss:", results[0])
# print("Validation accuracy:", results[1])


In [ ]:
# import tensorflow as tf

# # Detect TPU
# try:
#     tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
#     print('Running on TPU ', tpu.master())
# except ValueError:
#     raise BaseException('ERROR: Not connected to a TPU runtime; please check your settings.')

# # Connect to the TPU cluster
# tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)

# # Create TPU strategy
# strategy = tf.distribute.TPUStrategy(tpu)

# with strategy.scope():
#     # Your model and training code goes here
#     trainer.train()
